In [3]:
import sys
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

import torch
from torchvision.io import read_image
from torchvision import models, transforms, datasets

In [16]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("nonone")
    sys.exit()

#model import
model = "./opencv_face_detector_uint8.pb" # 파라미터가 들어가있는 모델
config = "./opencv_face_detector.pbtxt"

face_net = cv2.dnn.readNet(model, config)

if face_net.empty():
    print("novideo")
    sys.exit()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    #image resize
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123), swapRB=False)
    face_net.setInput(blob)
    out = face_net.forward() #face의 좌표들이 나옴옴

    detection = out[0, 0] # 2개의 더미index가 있어서 걷어 내고 나면 200 * 7
    h, w = frame.shape[:2]

    threshold = 0.5
    for i in range(len(detection)):
        confidence = detection[i, 2]


        if confidence > threshold:
            x1 = int(detection[i, 3]*w) # 0~1사이
            y1 = int(detection[i, 4]*h)
            x2 = int(detection[i, 5]*w)
            y2 = int(detection[i, 6]*h)   

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)   
            label = f"face: {confidence*100:.2F}%"  
            cv2.putText(frame, label, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255),1, cv2.LINE_AA )    

    cv2.imshow("frame", frame)

    if cv2.waitKey(30) == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [14]:
#dir(models.detection)

weights = models.detection.SSD300_VGG16_Weights.COCO_V1 # 모델 불러오기 COCO dataset
model = models.detection.ssd300_vgg16(weights = weights)

model.eval()

SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=